In [120]:
import requests
import json
import math

## Querying the API for bus locations and stops

In [7]:
start_time = 1539490442179
end_time   = 1539490499179
routes = ["14","14R","49"]

In [14]:
def list_string(arr):
    return f"{arr}".replace("'",'"')

In [24]:
query = f"""{{
  trynState(agency: "muni", startTime: "{start_time}", endTime: "{end_time}", routes: {list_string(routes)}) {{
    agency
    startTime
    routes {{
      rid
      stops {{
        sid
        name
        lat
        lon
      }}
      routeStates {{
        vtime
        vehicles {{
          vid
          lat
          lon
          heading
        }}
      }}
    }}
  }}
}}
""" # Braces need to be doubled for f-string

In [27]:
query_url = "https://06o8rkohub.execute-api.us-west-2.amazonaws.com/dev/graphql?query="+query
r = requests.get(query_url)

### Parsing the returned data

In [44]:
data = json.loads(r.text)
route_list = data['data']['trynState']['routes']

In [57]:
route_list[0]['routeStates'][1]['vehicles'][4]

{'heading': 165, 'lat': 37.755463, 'lon': -122.418823, 'vid': '7257'}

In [61]:
route_list[0]['stops'][8]

{'lat': 37.78075,
 'lon': -122.4090999,
 'name': 'Mission St & 6th St',
 'sid': '5538'}

## Function to find the closest stop to a bus

In [80]:
def get_dist(loc1,loc2,lat_weight=1,lon_weight=1):
    return abs(loc1['lat']-loc2['lat'])/lat_weight + abs(loc1['lon']-loc2['lon'])/lon_weight

In [81]:
def get_closest(loc,stop_list,lat_weight=1,lon_weight=1):
    # Running minimum distance
    running_min=500 #Dumb high number

    for stop in stop_list:
        current_min=get_dist(loc,stop,lat_weight,lon_weight)
        if current_min < running_min:
            running_min = current_min
            running_best_stop = stop
            
    return running_best_stop

In [86]:
lat_scale = math.cos(37.7*math.pi/180)

In [92]:
bus_1 = route_list[0]['routeStates'][1]['vehicles'][2]
stops_14 = route_list[0]['stops']

get_closest(bus_1, stops_14, lat_scale)

{'lat': 37.72635,
 'lon': -122.4336499,
 'name': 'Mission St & Francis St',
 'sid': '5590'}